# Stack — Diffusion Policy Training

Train the diffusion policy (ResNet18 + ConditionalUnet1D) on GPU.

**Runtime:** Change to GPU via `Runtime > Change runtime type > T4 GPU` (or A100 with Colab Pro).

In [ ]:
# Verify GPU
import torch
assert torch.cuda.is_available(), "No GPU! Change runtime type."
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

In [ ]:
# Clone repo (set to private? use token if needed)
!git clone https://github.com/corneliusgruss/stack.git
%cd stack

In [ ]:
# Install deps (most already in Colab)
!pip install -q diffusers omegaconf wandb

In [ ]:
# Upload session data
# Option A: Upload zip from local machine
from google.colab import files
uploaded = files.upload()  # Upload sessions.zip

!mkdir -p data/raw/real
!unzip -q sessions.zip -d data/raw/real/

In [ ]:
# Option B: Mount Google Drive (better for large datasets)
# from google.colab import drive
# drive.mount('/content/drive')
# !ln -s /content/drive/MyDrive/stack_data data/raw/real

In [ ]:
# Quick sanity check: train on synthetic data first
!python -m stack.data.synthetic --output data/raw/synthetic --num-sessions 20

In [ ]:
# Train on synthetic (pipeline validation)
!python -m stack.scripts.train \
    --config configs/default.yaml \
    --data-dir data/raw/synthetic \
    --output-dir outputs/synthetic_test

In [ ]:
# Train on real data
!python -m stack.scripts.train \
    --config configs/default.yaml \
    --data-dir data/raw/real \
    --output-dir outputs/real \
    --wandb

In [ ]:
# Evaluate
!python -m stack.scripts.eval \
    --checkpoint outputs/real/checkpoint_best.pt \
    --data-dir data/raw/real

In [ ]:
# Download checkpoint
from google.colab import files
files.download('outputs/real/checkpoint_best.pt')